# Astro Jeopardy
##### By Emily Valenta, Gabriella Allen, and Isabelle Tardivon
Below is our main code, mainly dealing with the front end of the game. This is where you'll find the board, board point values, daily double selection, as well as calling functions from other python files. 

The following cell block creates the main elements of the board: The point values, categories, random daily double assignment, and tracking whether or not a tile has been used. This generates multiple arrays that will later be used to generate the board tiles in `clue_object_setup.py`. 

**Daily Double:** The daily double takes a random integer from 1 to 30 and converts that number into a specific row and column on the game board. This tile is assigned as the daily double, and if chosen, will prompt the player to wager points. 

**beenUsed Array:** This array keeps track of which tile has already been used during the game. When false, the tile can be selected and used to earn points. When true, the tile cannot be chosen and prompts the player to choose a different tile. 

In [2]:
import numpy as np
#1. defines the six categories (exoplanets, galaxies, constellations, planets, moons, astronomers/physicists/astronomers) and point values (100,200,300,400,500)
categories = ["exoplanets", "galaxies", "constellations", "planets", "moons", "astronomers"]
# 5 rows = 
board_values = np.array([[100, 100, 100, 100, 100, 100], 
                         [200, 200, 200, 200, 200, 200], 
                         [300, 300, 300, 300, 300, 300], 
                         [400, 400, 400, 400, 400, 400], 
                         [500, 500, 500, 500, 500, 500] ])

# 2. Randomly pick a Daily Double location on the board using choice
dd_selection = np.random.choice(30)

# Convert that one number into specific row (0-4) and col (0-5)
dd_row = dd_selection // 6
dd_col = dd_selection % 6

print(f"Daily Double Location: Row {dd_row}, Col {dd_col} (Category: {categories[dd_col]})")

# 3. Initialize Board with placeholders (5 rows x 6 columns)

ddArray = np.array(["Q" for _ in range(30)])
ddArray = ddArray.reshape(5, 6)
ddArray[dd_row][dd_col] = "DD" # Mark Daily Double

# 4. Display Board
print("\nJeopardy Board:")
print(ddArray)

list = []
for i in range(30):
    list.append(False)
beenUsed = np.array(list)
beenUsed = beenUsed.reshape(5, 6)
print(beenUsed)

Daily Double Location: Row 0, Col 0 (Category: exoplanets)

Jeopardy Board:
[['D' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']]
[[False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]]


## Board Setup

Below these import statements are the three main functions that set up the board:
- **create_answers():** From `AI_Setup.py`, this function takes `astro_jeopardy_answers.csv` and makes it into a one-dimensional array. Having the answers to each fact makes it easier to specify to the LLM to not include the answer in the generated clue later in the code. This is also useful for showing the player the correct answer when they get a clue incorrect. 
- **create_clues():** Also from `AI_Setup.py`, this function takes `astro_jeopardy_facts.csv` and makes it into a one-dimensional array. Then, using the `prompt_llm()` function, each fact is iterated through a loop that uses an LLM to produce a Jeopardy-style clue based on the given fact. We made sure to be specific with our prompt, telling it to only respond with the produced clue and to not include the answer (which is also iterated through `prompt_llm()` in the same loop using `zip()`). Both the answer and clue arrays are reshaped into 5 by 6 arrays to match the board. (We used flattened arrays for this function because it was easier to zip the two arrays together and iterate through them.)
- **generate_tiles():** This function comes from `clue_object_setup.py`. This python file sets up the main board made up of `Tile` objects. Each `Tile` contains a clue, answer, point value, whether or not it's the daily double, and if it's already been used. We will use these to call specific values stored in this object later on in the game. 

In [3]:
from clue_object_setup import Tile
from clue_object_setup import generate_tiles
from AI_Setup import create_clues
from AI_Setup import create_answers
from Play_Function import play
from AI_Setup import compare_answer
from player_object_setup import Player
from player_object_setup import startGame

answer = create_answers()
clues, answers = create_clues(answer)

board = generate_tiles(clues, answers, ddArray, board_values, beenUsed)



Successfully loaded Astronomy 1221 key
Successfully found .gitignore in the current directory
Confirmed that .gitignore has the .env exclusion
astro_jeopardy_answers.csv exists in the current directory.
astro_jeopardy_facts.csv exists in the current directory.
Correct
Clues generated successfully.
HD189733b


In [4]:


def assignClue(category, point_values):
    """
    Retrieves the chosen clue using the selected category and point_values. 
    Parameters: category (must be exoplanets, galaxies, constellations, solar system, moons, or astronomers) and 
    point_values (must be 100, 200, 300, 400, or 500)

    Error handling: If a category or point value is selected that isn't an option, play() in Play_Function.py handles 
    these errors and returns the player to category and point selection. 
    """


    if category == "exoplanets":
        board_column = 0
    elif category == "galaxies":
        board_column = 1
    elif category == "constellations":
        board_column = 2
    elif category == "solar system":
        board_column = 3
    elif category == "moons":
        board_column = 4
    elif category == "astronomers":
        board_column = 5

    if point_values == "100":
        board_row = 0
    elif point_values == "200":
        board_row = 1
    elif point_values == "300":
        board_row = 2
    elif point_values == "400":
        board_row = 3
    elif point_values == "500":
        board_row = 4

    return board[board_row][board_column]

def getPlayerAnswer(chosenTileAnswer, chosenTileClue):
    userAnswer = input(f"{chosenTileClue}")
    print(f"Your answer: {userAnswer}")
    output = compare_answer(chosenTileAnswer, userAnswer)
    return output

def editPoints(chosenTile, correctness, player):
    if correctness == "Correct":
        player.points += chosenTile.pointValue
        chosenTile.beenUsed = True
    elif correctness == "Incorrect":
        player.points -= chosenTile.pointValue
        print(f"The correct answer is: {chosenTile.answer}")
        chosenTile.beenUsed = True
    else:
        player.points = player.points
    
    return player.points



In [5]:


while True:

    player1 = startGame()
    
    print(f"\n")

    while True:

        point_values, category = play()

        chosenTile = assignClue(category, point_values)

        while True: 

            if chosenTile.beenUsed:
                print("You have already selected this tile.")
                break
            elif (chosenTile.isDailyDouble == "D") & (player1.points > 0):
                wager = int(input("You have selected the Daily Double! How many points would you like to wager?"))
                if wager > player1.points:
                    print("You can't wager more than the amount of points you have!")
                elif wager <= player1.points:
                    chosenTile.pointValue = wager

            print(chosenTile.clue)
            correctness = getPlayerAnswer(chosenTile.answer, chosenTile.clue)

            if correctness == "Please respond with your answer formatted as a question.":
                continue
            else: 
                break

        currentPoints = editPoints(chosenTile, correctness, player1)

        print(f"Current Points: {currentPoints}")
        print(f"----------------")

        stop = input(f"You currently have {currentPoints} points. Press enter to continue, or type 'Y' to stop playing:")
        if stop == "Y":
            print(f"Thanks for playing!")
            break
        elif stop == "y":
            print(f"Thanks for playing!")
            break
        else:
            continue
    
    

    break



Hello, Welcome to AstroJeopardy! Please enter user information.

Host: Hi Isabelle! What are you studying as a student in Columbus?

Host: Oops, I didn't catch that! What are you studying as a student in Columbus?

Host: Physics, that's fascinating! What got you interested in studying physics?

Host: Space is truly amazing! Do you have a favorite planet or celestial object?


This constellation, shaped like a distinctive “W,” represents a queen from Greek mythology who was punished by Poseidon for her vanity about her daughter’s beauty.
Your answer: What is Cassieopeia?
Correct
Current Points: 300
----------------
Please select a valid category.
Please select a valid category.
Please select a valid category.
Please select a valid category.
This spiral galaxy, the closest to the Milky Way, was first recorded in 964 by Persian astronomer Abd al-Rahman al-Sufi and is also known as Messier 31.
Your answer: What is Andromeda?
Correct
Current Points: 500
----------------
Please select a vali